In [241]:
import numpy as np
import random
from matplotlib import pyplot as plt
from keras.datasets import fashion_mnist
from tqdm import tqdm
from numba import jit,cuda
import math

In [ ]:
class PreProc:
    def __init__(self):
        (self.trainx,self.trainy),(self.testx, self.testy) = fashion_mnist.load_data()
    def visualize(self,n):
        for i in range(n):
            plt.subplot(330+1+i) # ask someone why??
            plt.imshow(self.trainx[i], cmap = plt.get_cmap('gray'))
        plt.show()
    def flatten(self):
        trainx_flattened = self.trainx
        testx_flattened = self.testx
        trainx_flattened.shape = (60000,784)
        testx_flattened.shape = (10000,784)
        return trainx_flattened,testx_flattened
    def getLabels(self):
        return self.trainy, self.testy
    def getInputSize(self):
        return len(self.trainx[0])

In [ ]:
data.getInputSize()

(784,)

In [268]:
class Functions:
    @staticmethod
    def sigmoid(input):
        return  1.0/(1.0+np.exp(-input))
    @staticmethod
    def softmax(input):
        return np.exp(input)/(np.sum(np.exp(input)))
    @staticmethod
    def onehot(input):
        result = np.zeros(10)
        result[input] = 1
        return result
    @staticmethod
    def crossEntropyLoss(y,yHat):
        loss = (-1/10.0) * np.sum(np.multiply(y, np.log(yHat)) + np.multiply((1 - y), np.log(1 - yHat)))
        return loss
    @staticmethod
    def derivative_sigmoid(input):
        return Functions.sigmoid(input)*(1-Functions.sigmoid(input))
    def plot(input):
        plt.plot(input)
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title("Loss over iterations")

In [276]:
class Algorithms:
    @staticmethod
    def ForwardProp(net, activate, output, inputLayer):
        L = len(net)-1
        a = []
        h = []
        weights = net[0][:,:len(net[0][0])-1]
        bias = net[0][:,len(net[0][0])-1]
        temp = np.matmul(weights,inputLayer)+bias
        a.append(temp)
        h.append(activate(a[0]))
        for k in range(1,L):
            weights = net[k][:,:len(net[k][0])-1]
            bias = net[k][:,len(net[k][0])-1]
            temp = np.matmul(weights,h[k-1])+bias
            a.append(bias + np.matmul(weights,h[k-1]))
            h.append(activate(a[k]))
        weights = net[L][:,:len(net[L][0])-1]
        bias = net[L][:,len(net[L][0])-1]
        temp = np.matmul(weights,h[L-1])+bias
        a.append(temp)
        h.append(output(a[L]))
        return a,h
    @staticmethod
    def BackProp(net, a, h, dataPoint, dataLabel):
        L = len(net)-1
        gradaL = -(Functions.onehot(dataLabel)-h[len(h)-1])
        gradient = np.zeros_like(net)
        for k in range(L,0,-1):
            gradW = np.outer(gradaL,h[k-1].T)
            gradB = gradaL
            gradB.resize((len(gradB),1))
            grad = np.append(gradW,gradB,axis=1)
            gradient[k] = grad

            gradhL_1 = np.matmul(np.transpose(net[k][:,len(net[k])-1]),gradaL)
            gradaL_1 = np.multiply(gradhL_1, Functions.derivative_sigmoid(a[k-1]))
            gradaL = gradaL_1
        gradW = np.outer(gradaL,dataPoint.T)
        gradB = gradaL
        gradB.resize((len(gradB),1))
        grad = np.append(gradW,gradB,axis=1)
        gradient[0] = grad
        return gradient
    def mGD(net, learningRate, train, label, activate, output):
        beta = 0.9
        prevGrad = np.zeros_like(net)
        for i in range(len(train)):
            a,h = Algorithms.ForwardProp(net, activate, output, train[i])
            gradient = Algorithms.BackProp(net, a, h, train[i], label[i])
            gradient += beta*prevGrad + learningRate*gradient
        net = net - gradient

In [66]:
len(h)

4

In [63]:
for i in range(L,0,-1):
    print(i, end = " ")

3 2 1 

4

In [158]:
#The class of FeedForwardNeuralNetwor

class FFNet:
    #constructor
    def __init__(self,number_of_hidden_layers, number_of_inputs, number_of_outputs):
        self.number_of_inputs = number_of_inputs
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_outputs = number_of_outputs
        self.input = [0 for i in range(number_of_inputs)]
        self.output = [0 for i in range(10)]
        self.hidden = []
        #self.hidden.append(np.random.random((number_of_inputs+1)))
    
    #Method for creating layers
    def addHiddenLayer(self,number_of_neurons):
        if(len(self.hidden) == 0):
            temp_weights = np.random.random((number_of_neurons, self.number_of_inputs+1)) #The +1 is for biases.
        else:
            prev_neurons = len(self.hidden[len(self.hidden) - 1])
            temp_weights = np.random.random((number_of_neurons, prev_neurons + 1)) # The +1 is for biases.
        temp_weights = temp_weights/np.linalg.norm(temp_weights)
        self.hidden.append(temp_weights)
    
    def addOutputLayer(self, number_of_outputs):
        if(len(self.hidden) == 0):
            #print("number of inputs: "+str(self.number_of_inputs))
            temp_weights = np.random.random((number_of_outputs, self.number_of_inputs)) #Bias not needed for the output layer.
        else:
            prev_neurons = len(self.hidden[len(self.hidden) - 1])
            temp_weights = np.random.random((number_of_outputs, prev_neurons + 1)) #Bias not needed for the output layer.
        temp_weights = temp_weights/np.linalg.norm(temp_weights)
        self.hidden.append(temp_weights)

    def solidify(self):
        self.network = np.array(self.hidden)

    def getNetwork(self):
        return self.network
    
    def ForwardProp(self, activate, output, inputLayer):
        return Algorithms.ForwardProp(self.network, activate, output, inputLayer)
    
    def lossCalc(self, lossFunction, Y):
        predY = self.historyA[(len(self.historyA)-1)]
        return lossFunction(Y,self.predY)

    def BackProp(self, a, h, dataPoint, dataLabel):
        return Algorithms.BackProp(self.network, a, h, dataPoint, dataLabel)

    def gradientDescent(self, learningRate, lossFunction, activate, output, dataPoints, dataLabels):
        losses = []
        for i in range(5):
            gradient = 0.0
            loss = 0
            for index in tqdm(range(len(dataPoints))):
                a,h = Algorithms.ForwardProp(self.network, activate, output, dataPoints[index])
                predY = h[(len(h)-1)]

                loss += Functions.crossEntropyLoss(dataLabels[index], predY)
                gradient+= Algorithms.BackProp(self.network, a, h, dataPoints[index], dataLabels[index])
            losses.append(loss)
            #gradient /= 60000
            self.network = self.network - learningRate * gradient
        return self.network, losses
    def stochGradientDescent(self, learningRate, lossFunction, activate, output, dataPoints, dataLabels):
        #losses = []
        net = self.network
        for i in range(3):
            for i in tqdm(range(len(dataPoints))):
                a,h = Algorithms.ForwardProp(net,Functions.sigmoid, Functions.softmax, dataPoints[i])
                gradient = Algorithms.BackProp(net, a,h,dataPoints[i], dataLabels[i])
                net = net - 0.1*gradient
        return self.network
    
    def batchGD(self, learningRate, lossFunction, activate, output, dataPoints, dataLabels, batchSize = 32):
        indices = np.arrange(60000)
        for epoch in range(15):
            np.shuffle(indices)
            for i in range(batchSize):
                
    def evaluateNetwork(self,test_x, test_y):
        num_acc = 0
        for i in range(len(test_x)):
            a,h = Algorithms.ForwardProp(self.network, Functions.sigmoid, Functions.softmax, test_x[i])
            h = np.array(h)
            predY =   np.argmax(h[len(h)-1])
            if test_y[i] == predY:
                num_acc+=1
        print(num_acc/len(test_y))

In [ ]:
class Parallelizer:
    @staticmethod
    @cuda.jit
    def entireDataSet(net, dataPoints, dataLabels, activate, output, learningRate, answer, losses):
        index = cuda.grid()
        a,h = ParallelAlgorithms.ForwardProp(net, activate, output, dataPoints[index])
        predY = h[(len(h)-1)]

        loss = Functions.crossEntropyLoss(dataLabels[index], predY)
        losses[index] = (loss)
        gradient = ParallelAlgorithms.BackProp(net, a, h, dataPoints[index], dataLabels[index])
        cuda.atomic.add(answer,0,gradient,answer)


In [277]:
if __name__ == '__main__':
    data = PreProc()
    #data.visualize(5)
    train_x, test_x = data.flatten()
    train_y, test_y = data.getLabels()
    trainx = train_x/255
    testx = test_x/255
    neuralNet = FFNet(0,data.getInputSize(), 10)
    neuralNet.addHiddenLayer(128)
    neuralNet.addOutputLayer(10)
    neuralNet.solidify()
    net = neuralNet.getNetwork()

<ipython-input-158-6d2012216449>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.network = np.array(self.hidden)


In [261]:
trainx

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [228]:
a,h = Algorithms.ForwardProp(net, Functions.sigmoid, Functions.softmax, train_x[500])

In [222]:
np.argmax(np.array(h[L]))

2

In [278]:
batchSize = 32
gradient = np.zeros_like(net)
lossTrack = []
for epoch in tqdm(range(15)):
    indices = np.arange(len(trainx))
    np.random.shuffle(indices)
    batchX = trainx[indices]
    batchY = train_y[indices]
    for i in range(math.ceil(len(trainx)/batchSize)):
        trainer = batchX[i*batchSize:i*batchSize+batchSize]
        labeler = batchY[i*batchSize:i*batchSize+batchSize]
        batchLoss = 0.0
        for data in range(batchSize):
            a,h = Algorithms.ForwardProp(net, Functions.sigmoid, Functions.softmax, trainer[data])
            currGrad = Algorithms.BackProp(net, a, h, trainer[data], labeler[data])
            batchLoss += Functions.crossEntropyLoss(Functions.onehot(labeler[data]), h[-1])
            gradient += currGrad
        batchLoss /= 32
        gradient /= 32
        net = net - 0.1*gradient 
        lossTrack.append(batchLoss)
    print("The loss after this epoch is: "+ str(batchLoss))
        #print(net[0][23][34], end = " "),

  7%|▋         | 1/15 [00:52<12:18, 52.75s/it]

The loss after this epoch is: 0.25368447447464426


 13%|█▎        | 2/15 [01:44<11:18, 52.20s/it]

The loss after this epoch is: 0.25368243603437446


 20%|██        | 3/15 [02:36<10:25, 52.17s/it]

...

The loss after this epoch is: 0.1564776869395773


 93%|█████████▎| 14/15 [12:19<00:54, 54.08s/it]

The loss after this epoch is: 0.17367634158021686


100%|██████████| 15/15 [13:12<00:00, 52.82s/it]

The loss after this epoch is: 0.1151368321699927


In [177]:
net

array([array([[-1.69749915e+01, -1.24787929e+02, -6.52443942e+02, ...,
               -1.80137723e+04, -1.55251033e+03, -5.60673889e+06],
              [-1.69772276e+01, -1.24783938e+02, -6.52447094e+02, ...,
               -1.80137747e+04, -1.55251533e+03, -5.60673889e+06],
              [-1.69772186e+01, -1.24785388e+02, -6.52444464e+02, ...,
               -1.80137759e+04, -1.55251294e+03, -5.60673889e+06],
              ...,
              [-1.69768807e+01, -1.24784904e+02, -6.52445089e+02, ...,
               -1.80137719e+04, -1.55251244e+03, -5.60673889e+06],
              [-1.69748415e+01, -1.24786799e+02, -6.52443977e+02, ...,
               -1.80137749e+04, -1.55251116e+03, -5.60673889e+06],
              [-1.69798755e+01, -1.24784666e+02, -6.52443596e+02, ...,
               -1.80137722e+04, -1.55251218e+03, -5.60673889e+06]])   ,
       array([[-0.29924753, -0.30440764, -0.29254498, ..., -0.29417291,
               -0.29298389, -0.6355123 ],
              [-0.29778866, -0.299

In [117]:
gradient = neuralNet.BackProp(a,h,train_x[1], train_y[1])


In [118]:
net = net - gradient*0.1

In [119]:
net

array([array([[ 0.00212451,  0.00348055,  0.00273668, ...,  0.00160233,
                0.00401092, -0.10310577],
              [ 0.0053647 ,  0.00488688,  0.00141693, ...,  0.00345387,
                0.00127243, -0.10183299],
              [ 0.00389507,  0.00445134,  0.00076357, ...,  0.00094713,
                0.00404774, -0.10074752],
              ...,
              [ 0.00471922,  0.00443626,  0.0050084 , ...,  0.0010763 ,
                0.00512554, -0.10414192],
              [ 0.00516156,  0.00476448,  0.00393312, ...,  0.00430722,
                0.00099776, -0.10057883],
              [ 0.00329868,  0.00151445,  0.00457194, ...,  0.00074116,
                0.00449557, -0.10262027]])                             ,
       array([[-0.00099111, -0.00721432, -0.00519615, ..., -0.00653088,
               -0.00090633, -0.00623443],
              [-0.00418149, -0.00042853, -0.00603201, ..., -0.00721158,
                0.00273719, -0.00376062],
              [-0.0071174 ,  0.0004852

In [169]:
np.argmax(np.array(h[L]))

1

In [150]:
while np.argmax(np.array(h[L])):
    for i in range(10):
        a,h = Algorithms.ForwardProp(net,Functions.sigmoid, Functions.softmax, train_x[4])
        gradient = neuralNet.BackProp(a,h,train_x[4], train_y[4])
        net = net - 0.1*gradient
        print(np.argmax(np.array(h[L])), end = ", "),

3, 3, 0, 

In [149]:
train_y[4]

0

In [279]:
num_acc = 0
for i in range(len(test_x)):
    a,h = Algorithms.ForwardProp(net, Functions.sigmoid, Functions.softmax, test_x[i])
    h = np.array(h)
    predY =   np.argmax(h[len(h)-1])
    print(predY)
    if test_y[i] == predY:
        num_acc+=1
print(num_acc/len(test_y), end = " ")


8
4
8
9
3
8
0
9
4
9
0
9
2
9
8
0
7
9
9
4
8
9
1
0
8
9
8
0
8
9
8
0
0
1
5
4
0
1
9
9
7
4
8
0
9
2
4
8
7
9
8
9
0
9
7
8
8
9
9
3
8
8
4
3
9
3
1
0
9
8
2
8
8
9
8
8
8
9
9
0
0
9
8
2
1
8
0
3
8
7
0
8
4
9
1
1
2
0
8
9
8
9
8
8
0
1
4
0
1
8
0
3
8
8
9
3
8
8
8
3
9
1
0
1
8
8
4
0
9
3
8
0
1
8
8
1
0
8
9
0
0
9
8
3
8
8
9
8
9
8
0
8
0
9
9
9
9
9
4
8
8
2
0
1
9
2
9
8
3
1
8
8
8
8
9
0
4
4
2
1
3
8
0
3
9
9
7
9
1
0
4
4
9
0
4
3
1
8
4
8
2
8
9
3
1
8
1
9
2
9
0
2
9
1
8
4
9
8
8
0
9
8
8
8
1
0
4
9
4
9
8
5
9
8
8
0
8
5
9
9
9
8
2
8
2
1
3
3
9
8
9
4
0
8
3
2
8
1
0
4
1
0
7
4
9
9
2
8
9
4
8
9
1
2
1
9
4
8
8
8
0
2
0
0
1
2
4
7
0
1
5
8
2
7
1
9
8
8
1
2
8
0
9
3
8
9
0
8
8
2
7
9
9
4
2
9
1
9
1
8
2
3
8
8
4
0
4
8
8
2
9
3
8
8
0
1
9
9
8
3
8
3
4
8
0
2
1
3
3
0
9
0
8
9
8
9
3
8
9
9
4
8
8
0
1
4
4
0
9
8
1
2
1
0
3
2
8
8
8
0
8
3
4
4
8
8
9
4
8
2
4
7
0
1
8
0
9
9
0
0
3
8
1
1
2
4
2
9
8
1
0
1
8
8
8
8
0
4
0
4
9
3
8
8
4
8
9
2
8
8
4
4
0
8
8
8
2
8
9
2
9
7
9
2
1
0
4
1
9
8
1
8
3
9
8
8
9
8
1
8
2
0
0
0
1
0
4
3
8
0
4
2
8
3
3
8
0
3
0
9
1
8
3
1
8
8
8
4
9
0
3
8
8
1
0
9
0
9
8
1

In [ ]:
len(test_y)

10000

In [ ]:
gradient[0].shape

(20, 785)

In [ ]:
for i in gradient[0]:
    print(i, end = " ")

In [ ]:
gradient[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
gradaL = -(Functions.onehot(train_y[1])-h[len(h)-1])

In [ ]:
gradhL_1 = np.matmul(np.transpose(net[(len(net)-1)]),aL)

In [ ]:
gradaL_1 = np.multiply(net[len(net)-1][:,:len(net[len(net)-1][0])-1], Functions.derivative_sigmoid(a[len(net)-2]))

In [ ]:
gradW = np.outer(gradaL,h[len(net)-2].T)
gradB = gradaL


In [ ]:
gradB.resize((len(gradB),1))

In [ ]:
gradB.shape

(10, 1)

In [ ]:
np.append(gradW,gradB.resize((10,1)),axis=1)

array([[-0.49974422, -0.49987732, -0.50045317, -0.49836884, -0.50235192,
        -0.50235109, -0.49928474, -0.49920566, -0.49775972, -0.49974985,
        -0.49777699, -0.50029926, -0.49888012, -0.50086685, -0.50025429,
        -0.49987588, -0.49888532, -0.49982542, -0.50135421, -0.50049566,
        -0.89963249],
       [ 0.05646266,  0.0564777 ,  0.05654276,  0.05630726,  0.05675728,
         0.05675719,  0.05641074,  0.05640181,  0.05623844,  0.05646329,
         0.05624039,  0.05652537,  0.05636503,  0.0565895 ,  0.05652029,
         0.05647753,  0.05636562,  0.05647183,  0.05664456,  0.05654756,
         0.10164328],
       [ 0.05479879,  0.05481338,  0.05487653,  0.05464797,  0.05508473,
         0.05508464,  0.05474841,  0.05473974,  0.05458118,  0.05479941,
         0.05458308,  0.05485965,  0.05470404,  0.05492189,  0.05485472,
         0.05481323,  0.05470461,  0.05480769,  0.05497533,  0.05488119,
         0.09864801],
       [ 0.05828847,  0.058304  ,  0.05837116,  0.05812805

In [ ]:
gradaL.shape

(10,)

In [ ]:
a[len(net)-2].shape

(20,)

In [ ]:
weights = net[0][:,:len(net[0][0])-1]
bias = net[0][:,len(net[0][0])-1]
temp = np.matmul(weights,train_x[0])+bias
temp = temp/np.linalg.norm(temp)
a = []
a.append(temp)
h = []
h.append(Functions.sigmoid(a[0]))

In [ ]:
weights = net[L][:,:len(net[L][0])-1]
bias = net[L][:,len(net[L][0])-1]
temp = np.matmul(weights,h[0])+bias
temp = temp/np.linalg.norm(temp)

In [ ]:
L = len(net)-1

In [ ]:
weights.shape

(10, 20)

In [ ]:
#The class of FeedForwardNeuralNetwor

class FFNet:
    #constructor
    hidden = []
    input = []
    output = []
    def __init__(self,number_of_hidden_layers, number_of_inputs, number_of_outputs):
        self.number_of_inputs = number_of_inputs
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_outputs = number_of_outputs
        #At the same time, the layers input layers mus also be initialized.

        input = [0 for i in range(number_of_inputs)]
        output = [0 for i in range(number_of_outputs)]
        hidden = [[]]

        #input and output layers are nothing but simple lists
    
    #Method for creating layers
    def add_hidden_layer(number_of_neurons):
        temp_weights = [0 for i in range(number_of_neurons+1)] #The +1 is for bias values
        hidden.append(temp_weights)
    
    def backward_propagate(a,h, pred_y):
        delthet[L] = -(exp(y) - pred_y) #with respect to output layer
        for k in range(0,L-1,-1):
            delthetw = np.matmul(delthet[k], h[k-1].T)
            delthetb = delthet[k]
            deltheth = np.matmul(weights[k].T, delthet[k])
            delthet[k-1] = hadamard(deltheth, preac(a)) 

    def forward_propagate():
        #here, we are calculating the preactivations and activations.
        #we then store them in an array and return it.
        
        for k in range(number_of_levels-1):
            a[k] = biases[k] + np.matmul(weights[k], h[k-1])
            h[k] = g(a[k])
        a[number_of_levels-1] = biases[number_of_levels] + np.matmul(weights[number_of_levels],h[number_of_levels-1])
        pred_y = output(a[number_of_levels-1])
        return a,h, pred_y


    def gradient_descent():
        a,h, pred_y = forward_propagate()
        delthet = backward_propagate(a,h, pred_y)
        thet += delthet

    def fit(dataset):
        for x,y in dataset:
            loss = forward(x,y)
            delthet = backward(loss)
            thet += learn_rate*delthet
    